In [49]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 12 10:04:40 2020

@author: alexe
"""

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import time
import datetime

#чтение данных
features = pd.read_csv('data/features.csv', index_col='match_id')
features_test = pd.read_csv('data/features_test.csv', index_col='match_id')

#количество признаков определяем по тестовой выборке, первый столбец не считаем - это номера игр
N = np.shape(features_test)[1] - 1

Y = features['radiant_win'] #целевая переменная - исход игры, 1 при победе radiant, 0 при победе ward

#проверяем наличие пропусков
Rows = np.shape(features.iloc[:,0])[0]
for i in range(0,np.shape(features)[1]):
    COUNT = features.iloc[:,i].count()
    if Rows != COUNT:
        print('В столбце',features.iloc[0:0,i],'тренировочной выборки имеется',Rows-COUNT,'пропусков из',Rows,'элементов')

Rows_test = np.shape(features_test.iloc[:,0])[0]
for i in range(0,N+1):
    COUNT = features_test.iloc[:,i].count()
    if Rows_test != COUNT:
        print('В столбце',features_test.iloc[0:0,i],'тестовой выборки имеется',Rows_test-COUNT,'пропусков из',Rows_test,'элементов')

#заполняем пропуски, создаём массивы данных отбрасываем ненужное
#(номера матчей, а также лишние признаки в тренировочной выборке - они идут после основных)
FILL = 0
X = features.fillna(FILL).iloc[:,1:N+1]
X_test = features_test.fillna(FILL).iloc[:,1:N+1]

#кросс-валидация
kf = KFold(n_splits=5,shuffle=True,random_state=1)

#бустинг
clf1 = [0]*3
clf2 = [0]*3
ACCURACY1 = [0] * 3
ACCURACY2 = [0] * 3
NTREES = [10,20,30]
for i in range(0,3):
    clf1[i] = GradientBoostingClassifier(n_estimators=NTREES[i], random_state=1)
    start_time = datetime.datetime.now()
    ACCURACY1[i] = cross_val_score(clf1[i], X, Y, cv=kf, scoring='roc_auc').mean()
    print('Время работы:', datetime.datetime.now() - start_time)
    print('Точность градиентного бустинга с',NTREES[i],'деревьями в метрике AUC-ROC',ACCURACY1[i])
    clf2[i] = AdaBoostClassifier(n_estimators=NTREES[i], random_state=1)
    start_time = datetime.datetime.now()
    ACCURACY2[i] = cross_val_score(clf2[i], X, Y, cv=kf, scoring='roc_auc').mean()
    print('Время работы:', datetime.datetime.now() - start_time)
    print('Точность адаптивного бустинга с',NTREES[i],'деревьями в метрике AUC-ROC',ACCURACY2[i])

#логистическая регрессия
scaler = StandardScaler()
Xscaled = scaler.fit_transform(X)
ACCURACY3 = [0] * 10
C = [.00001, .001, .01, .1, 1, 10, 100, 1000, 10000, 100000]
clf3 = [0]*10
for i in range(0,10):
    clf3[i] = LogisticRegression(C=C[i], penalty='l2', random_state=1)
    start_time = datetime.datetime.now()
    ACCURACY3[i] = cross_val_score(clf3[i], Xscaled, Y, cv=kf, scoring='roc_auc').mean()
    print('Время работы:', datetime.datetime.now() - start_time)
    print('Точность логистической регрессии с L2-регуляризацией C=',C[i],'в метрике AUC-ROC',ACCURACY3[i])

#удаление категориальных признаков
X2 = X * 1
del X2['lobby_type']
for p in range(1,6):
    del X2['r'+str(p)+'_hero']
    del X2['d'+str(p)+'_hero']
scaler2 = StandardScaler()
X2scaled = scaler2.fit_transform(X2)
ACCURACY4 = [0] * 10
clf4 = [0]*10
for i in range(0,10):
    clf4[i] = LogisticRegression(C=C[i], penalty='l2', random_state=1)
    start_time = datetime.datetime.now()
    ACCURACY4[i] = cross_val_score(clf4[i], X2scaled, Y, cv=kf, scoring='roc_auc').mean()
    print('Время работы:', datetime.datetime.now() - start_time)
    print('Точность логистической регрессии с L2-регуляризацией C=',C[i],'при удалённых категориальных признаках в метрике AUC-ROC',ACCURACY4[i])

#работа с "мешком слов" для героев
heroes = np.unique([X['r1_hero'],X['r2_hero'],X['r3_hero'],X['r4_hero'],X['r5_hero'],X['d1_hero'],X['d2_hero'],X['d3_hero'],X['d4_hero'],X['d5_hero']])
heroesnum = np.amax([X['r1_hero'],X['r2_hero'],X['r3_hero'],X['r4_hero'],X['r5_hero'],X['d1_hero'],X['d2_hero'],X['d3_hero'],X['d4_hero'],X['d5_hero']])
heroes_pick = np.zeros((Rows, heroesnum))
start_time = datetime.datetime.now()
for i, match_id in enumerate(X.index):
    for p in range(1,6):
        heroes_pick[i, X['r'+str(p)+'_hero'][match_id]-1] = 1
        heroes_pick[i, X['d'+str(p)+'_hero'][match_id]-1] = -1
i = 0
currenthero = 1
while i < heroesnum:
    if not (currenthero in heroes):
        #герой с данным номером никогда не выбирался или вовсе не существует
        #данный признак полностью бесполезен, исключаем
        heroesnum -= 1
        heroes_pick = np.delete(heroes_pick,i,1)
    else:
        i += 1
    currenthero += 1
print('Время создания "мешка слов":', datetime.datetime.now() - start_time)
print('Количество героев',heroesnum)
X3 = np.hstack((X2,heroes_pick))
scaler3 = StandardScaler()
X3scaled = scaler3.fit_transform(X3)
ACCURACY5 = [0] * 10
clf5 = [0]*10
for i in range(0,10):
    clf5[i] = LogisticRegression(C=C[i], penalty='l2', random_state=1)
    start_time = datetime.datetime.now()
    ACCURACY5[i] = cross_val_score(clf5[i], X3scaled, Y, cv=kf, scoring='roc_auc').mean()
    print('Время работы:', datetime.datetime.now() - start_time)
    print('Точность логистической регрессии с L2-регуляризацией C=',C[i],'при преобразованных в "мешок слов" категориальных признаках в метрике AUC-ROC',ACCURACY5[i])

print('Лучший градиентный бустинг с числом деревьев',NTREES[np.argmax(ACCURACY1)],'точность',np.amax(ACCURACY1))
print('Лучший адаптивный бустинг с числом деревьев',NTREES[np.argmax(ACCURACY2)],'точность',np.amax(ACCURACY2))
print('Лучшая логистическая регрессия C=',C[np.argmax(ACCURACY3)],'точность',np.amax(ACCURACY3))
print('Лучшая логистическая регрессия без категориальных признаков C=',C[np.argmax(ACCURACY4)],'точность',np.amax(ACCURACY4))
print('Лучшая логистическая регрессия с категориальными признаками в "мешке слов" C=',C[np.argmax(ACCURACY5)],'точность',np.amax(ACCURACY5))

#обработка тестовой выборки
X2_test = X_test * 1
del X2_test['lobby_type']
for p in range(1,6):
    del X2_test['r'+str(p)+'_hero']
    del X2_test['d'+str(p)+'_hero']
heroes_pick_test = np.zeros((Rows_test, np.amax(heroes)))
start_time = datetime.datetime.now()
for i, match_id in enumerate(X_test.index):
    for p in range(1,6):
        heroes_pick_test[i, X_test['r'+str(p)+'_hero'][match_id]-1] = 1
        heroes_pick_test[i, X_test['d'+str(p)+'_hero'][match_id]-1] = -1
i = 0
currenthero = 1
while i < heroesnum:
    if not (currenthero in heroes):
        #герой с данным номером никогда не выбирался или вовсе не существует
        #данный признак полностью бесполезен, исключаем
        heroes_pick_test = np.delete(heroes_pick_test,i,1)
    else:
        i += 1
    currenthero += 1
print('Время создания "мешка слов" для тестовой выборки:', datetime.datetime.now() - start_time)
X3_test = np.hstack((X2_test,heroes_pick_test))
X3scaled_test = scaler3.transform(X3_test)

clf5[np.argmax(ACCURACY5)].fit(X3scaled, Y)
Y_test = clf5[np.argmax(ACCURACY5)].predict_proba(X3scaled_test)[:,1]
print('Минимальная и максимальная вероятность победы radiant на тестовой выборке',np.amin(Y_test),np.amax(Y_test))

f = open('test_result.csv','w')
f.write('match_id,radiant_win\n')
for i in range (0,Rows_test):
    f.write(str(features_test.index[i]) + ',' + str(Y_test[i]) + '\n')
f.close()

В столбце Series([], Name: first_blood_time, dtype: float64) тренировочной выборки имеется 19553 пропусков из 97230 элементов
В столбце Series([], Name: first_blood_team, dtype: float64) тренировочной выборки имеется 19553 пропусков из 97230 элементов
В столбце Series([], Name: first_blood_player1, dtype: float64) тренировочной выборки имеется 19553 пропусков из 97230 элементов
В столбце Series([], Name: first_blood_player2, dtype: float64) тренировочной выборки имеется 43987 пропусков из 97230 элементов
В столбце Series([], Name: radiant_bottle_time, dtype: float64) тренировочной выборки имеется 15691 пропусков из 97230 элементов
В столбце Series([], Name: radiant_courier_time, dtype: float64) тренировочной выборки имеется 692 пропусков из 97230 элементов
В столбце Series([], Name: radiant_flying_courier_time, dtype: float64) тренировочной выборки имеется 27479 пропусков из 97230 элементов
В столбце Series([], Name: radiant_first_ward_time, dtype: float64) тренировочной выборки имеетс

Время работы: 0:00:06.519851
Точность логистической регрессии с L2-регуляризацией C= 100000 при преобразованных в "мешок слов" категориальных признаках в метрике AUC-ROC 0.7517405396493333
Лучший градиентный бустинг с числом деревьев 30 точность 0.6895136679010616
Лучший адаптивный бустинг с числом деревьев 30 точность 0.6920558405032433
Лучшая логистическая регрессия C= 0.01 точность 0.7163005845105754
Лучшая логистическая регрессия без категориальных признаков C= 0.01 точность 0.7163299107224695
Лучшая логистическая регрессия с категориальными признаками в "мешке слов" C= 0.01 точность 0.7517860741867259
Время создания "мешка слов" для тестовой выборки: 0:00:02.603561
Минимальная и максимальная вероятность победы radiant на тестовой выборке 0.00841859257304547 0.9964525334561477
